## konlpy로 synop 명사 추출
## CountVectorizer, cosine_similarity

1. synop내 명사만으로는 유사도 계산 부정확
2. 제목, 태그, 대분류 완료 후  다시 계산
3. 불용어 제거
4. ~~다른 방송국 전체 데이터~~
5. 유사도 계산도 트레이닝이 필요한 것인가?
6. TfidfVectorizer은 다른 nb에 작성(-> TF, IDF 계산 식에 가중치 부여 가능여부?)
7. 각 idx에 추천 epi idx column 추가 후 csv파일 저장

In [255]:
import pandas as pd
import numpy as np
import unicodedata
import nltk
from nltk.corpus import stopwords
from konlpy.tag import *
#from ast import literal_eval

In [256]:
metadata = pd.read_csv('new_all_data_ver3.csv', encoding='cp949')
del metadata['synop']

In [257]:
metadata.head()

,idx,tv,title,epi,dates,links,vod_key,n_synop
0,1,SBS,SBS 스페셜,그랜드 슬램 대탐험 걸어서 지구 끝까지,2005.07.10,http://programs.sbs.co.kr/culture/sbsspecial/v...,"움직', '프로그램', '블리자드', '순간', '그랜드슬램'",스페셜 1편 그랜드 슬램 대탐험걸어서 지구 끝까지기획: 장광호김해영 촬영 (탐험대원...
1,2,SBS,SBS 스페셜,꿈을 쏘다! - 박주영 박지성,2005.07.17,http://programs.sbs.co.kr/culture/sbsspecial/v...,"젊은이', '그라운드', '박주영', '박지성', '선수', '실력', '축구', ...",국민 열광 두 젊은이 그라운드 승부사인 박주영 박지성 수 두 젊은이 번 난관 끝 포...
2,3,SBS,SBS 스페셜,메이드 인 코리아 1부 - 광복 60년 한국인들의 위상,2005.07.24,http://programs.sbs.co.kr/culture/sbsspecial/v...,"한국', '세계', '대표', '최고', '이들', '글라이더', '모습', '우리...",한국 해외 한국인 뭔가 책임감 끈기 인내 적응 갖다놔 그 국민성인거 세계최고층 빌딩...
3,4,SBS,SBS 스페셜,메이드 인 코리아 2부 - 한국천재들,2005.07.31,http://programs.sbs.co.kr/culture/sbsspecial/v...,"천재', '천재들', '세계', 'MP3', '김영세', 'PET', '연구소'",스페셜 메이드 코리아 2부 한국천재들21세기형 천재앞 세상 남 것 발견 것들 깨달아...
4,5,SBS,SBS 스페셜,메이드 인 코리아 3부 - 한국인의 손,2005.08.07,http://programs.sbs.co.kr/culture/sbsspecial/v...,"한국인', '세계'",한국인 손 불가능 벽 바 혁명 이루다세계 최초 인간 줄기세포 배양 성공 세계 황우석...


In [258]:
metadata['epi'][34]

'기적의 사막으로 오라 - 천지개벽 두바이'

In [259]:
metadata['n_synop'][755]

'아기 자연주의 출산이야기연출 신정현작가 전연옥옛날에는아무 병원 아기 생각 지금은아무 병원 이외 장소 아기 생각 세계 유례 정도로자연주 출산 비율 우리 대한민국정말아기 병원 걸까정말내진 관장 회음부 절개 임산부 3종 굴욕세트 경험 걸까제모 필요 걸까엄마 몸 아기 몸 영향 어떨까이런 질문 프로그램 시작 선택 자유 자연주의 출산건강 현미밥 챙겨먹고 채식위주 청정식단 인기 수술 디스크 치료 유행 시대 몸 약 칼 몸 시대 대한민국 산모들 출산당일 당연 듯 병원침대 몸 무통주사 생살 고통 속 당연 듯 회음부 절개 출산 아이 방법 한가지 밖 것 방법 것 《아 자연주의 출산이야기》 과거 99 1%미만 자연주의 출산사례들 우리 선택 자유 출산 이야기 환자 산모 굴욕출산 생명 탄생 경이로운 일 불구 출산 대부분 산모들 그 産苦 두려움 불안 산모 의료 개입 의사 권고 필요 약물 의 제왕절개 경우 병원 출산 일 무엇 출산 주체 산모 병원 시스템 객체 일 비일비재 자궁 이유 통 골절환자 배 산모 금식 과정 산모 비참 거 굴욕 3종 세트 내진(內診) 제모(除毛) 회음부 절개 이것 산모들 진저리 병원 출산 굴욕 3종 세트 산모들 회음부 절개 상처 출산 한참 후 통증 일반 산부인과 관례 행해지는회음부 절개” 것 답은아니다”다 미국은 6년 시술이다대한민국 99 병원출산자 선택은사 우리 어머니 어머니 어머니들 출산 자연 일 결혼 출산 당연 통과의례 그것 집 자연출산 제왕절개 병원 출산 산모들 마디 나 자연분 ”그러 진정한 자연출산 의료 개입 거 말 촉진제 무통주사 회음부 절개 등 일체 의료개입 산모 스스로 힘 의지 것 자연출산 우리나라 자연 출산율 1 미만 병원 출산 99 그 제왕절개 36 국 중 멕시코(39. 9 이탈리아(39. 7 다음 수치 뉴질랜드(14. 0 비교 3배 수치다(2010년 건강보험 심사 평가원 기준 모성애 세계 최강국 부족함 우리나라 산모들 자연출산 외면 것 출산 방법 산모 아기 건강 좌우한다인간 통 중 출산 통 세 번 만큼 고통 産苦 진통일까그 질문 답변 마디 아니다 

In [260]:
n_synoplist=metadata['n_synop']

In [261]:
n_synoplist[641]

'산 사람들 2자연식 비밀” 제작 S&K프로덕션 연출 정구익 글 구성 오정요 기획의도산 사람들 이번 자연식이다살기 산골생활 선택 사람들그들 식탁 뭔가 특별 것 있다말기암 선고 산 사람들 그 산 건강 그 비결 뭘까이번 그 식탁 극복 비밀 그 식탁 자연식 비밀 진행자 배우 강신일간암 고통 배우 강신일씨 진행 강신일씨 2007년 간암 수술 후충북 산골 6개월 건강 회복했다그 산 사람중 하나 주요내용 그 산 있었다산 사람들 1편 사람 1년 6개월 지금 그 있을까과연 그 산 극복 성공했을까다시 그 산 선고 이전 몸 자랑하며아예 전문농사꾼 이 안 암세포 지 있건만아무 생활 이 무엇 이 산 것일까이들 산 것 무엇 생활 시작 암환자들그들만 산생활 시작 사람들그들 말기암 선고 직후 산생활 선택 스무번 항암치료 죽음 문턱 사람들산생활 시작 후 건강 그 산 자신 것 목숨 도전한 그 생활그들 극복 수 그 공통점 하나 철저 자급자족산 사투 벌 그들 공통점 하나 농사 그 식재료 자신 지어먹는다 해산물 일부 제외 식재료 자 들 식재료들 그 믿음 확고 자신 건강 일등공신 식재료들 것.그들 그들 농사법 무엇인가그들 농사법 극복 비밀 그 식탁 다섯가지 비밀그들 식탁 몇 가지 패턴 현미밥 채식 위주 식사 기본 그 전부일까암환자 식단 한 전문가 그들 무심 식탁 기 영양소 균형 암환자 식단 황금비율그들은 황금비율 것일까그들 식재료 조리법 비밀 그 식재료 자신 텃밭 것 아니었다그들 식재료 하나 비밀 창고 진정한 자연식 것 그 식탁 자연식 비밀 다섯가지 효소 기적 효소 비밀그들 공통적 하나 음식 효소 그 집 효소창고 하나씩 그 그곳 자신 보물단지 말 그 효소 그 믿음 확고한데과연 믿음 근거 것일까일반 대학생 대상 그 효소 4일 동안 비교 세포 변화과연 그 믿음 근거 있었다산 사람들 그 자연식 비밀 다섯 가지 하나 비밀 병기 효소 그 식탁 분명 특별 것'

In [262]:
metadata['epi'][641]

'산에서 암을 이긴 사람들 2 - “자연식의 비밀”'

In [263]:
len(metadata['n_synop'])

1890

In [264]:
len(metadata['epi'])

1890

## 불용어 제거

In [265]:
'''def remove_stopwords(text):
    stops = ['이', '들', '것', '지', '수', '있는', '있습니다', '그', '년도', '에', '합니다', '하는', '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월', '저는', '없는', '것입니다', '등', '일', '많은', '이런', '것은', '왜', '같은', '없습니다', '위해', '한다']
    # Stopwords 불용어 제거
    meaningful_words = [word for word in text if not word in stops]
    return ' '.join(meaningful_words)
'''

"def remove_stopwords(text):\n    stops = ['이', '들', '것', '지', '수', '있는', '있습니다', '그', '년도', '에', '합니다', '하는', '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월', '저는', '없는', '것입니다', '등', '일', '많은', '이런', '것은', '왜', '같은', '없습니다', '위해', '한다']\n    # Stopwords 불용어 제거\n    meaningful_words = [word for word in text if not word in stops]\n    return ' '.join(meaningful_words)\n"

In [266]:
'''synop_list = []

for i in metadata['n_synop']:
    remove_stopwords(i)
'''

"synop_list = []\n\nfor i in metadata['n_synop']:\n    remove_stopwords(i)\n"

# CountVectorizer

In [267]:
from sklearn.feature_extraction.text import CountVectorizer

#1개 이하로 나오는 단어 제외
count = CountVectorizer(min_df=2)
#count_matrix = count.fit_transform(metadata)

In [268]:
count_matrix = count.fit_transform(metadata['n_synop'].values.astype('U'))

In [269]:
count_matrix.shape

(1890, 22006)

## 코사인 유사도 점수 계산

In [270]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [271]:
# index는 dataframe의 idx
indices = pd.Series(metadata.index, index=metadata['idx'])

In [272]:
def get_recommendations(idx, cosine_sim2=cosine_sim2):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim2[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
        
    return metadata['epi'].iloc[epi_indices]

### 나중에 epi & recommedation epi, index 한번에 출력 함수 만들기

In [297]:
metadata['epi'][1391]

'밥상, 상식을 뒤집다- 탄수화물의 경고 1부'

In [298]:
get_recommendations(1391)

1255         병원의 고백1부 - 너무나 친절한 의사들
1258          병원의 고백 2부. 의사에게 길을 묻다
1441                  젊음도 성형할 수 있나요
1337    암(癌) 전문의의 선택 진료실에서 못다 한 이야기
1450                        간호사의 고백
929           그녀, 뼈를 깎다 - 내 딸의 양악수술
261                      목숨 걸고 편식하다
1670                생존의 조건 - 권역외상센터
940            내 몸 속에는 100명의 의사가 있다
1641                    건강할 권리를 찾아서
Name: epi, dtype: object

In [299]:
get_recommendations(236)

324              최악의 시나리오 2부 - 온난화의 마지막 페이지
350     <아동범죄, 미스터리의 과학>1부 아이들은 왜 낯선 사람을 따라
57               29일째 날의 이스터 섬 - 과거로부터의 메시지
644                            몽골에서 지구를 지키다
525                          2009 영상기록 - 기후
1585      생과 사의 강 브라마푸트라 3부 분노의 강, 잃어버린 지평선
403                               대보초의 하얀죽음
352      <아동범죄, 미스터리의 과학>3부 우리아이 어떻게 지킬것인가?
1571                       달의 기적 1부 달이 차오른다
1276       생명 40억년의 비밀 5부 모든 것의 끝 혹은 시작, 멸종
Name: epi, dtype: object

In [300]:
get_recommendations(1)

380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

In [276]:
metadata['epi'][1]

'꿈을 쏘다! - 박주영 박지성'

In [277]:
metadata['n_synop'][1]

'국민 열광 두 젊은이 그라운드 승부사인 박주영 박지성 수 두 젊은이 번 난관 끝 포기 노력 연습 실력 공통점 공 마음 일 거 당연 일 박주영 신드롬 명승부 명장면 월드컵 예선 청소년축구대회 그 명승부 골 국민들 환호 이후 박주영 소재 패러디 인터넷 등장 박주영 응원가 것 그 리그 서울 관중수 예전 약 두 배 이 박주영 반응 마디 쿨하 주위 반응 아랑 곳하지 자기 일 전형적 신세대 놀이축구 성공 원 그 축구 운동장 엄마 시장 친구 때 축구공 환상적 그 드리볼 때 축구화 맨발 축구 덕분 실력 목표 달성 축구 것 세상 일 축구 축구만 생각 박주영 이전 선수들 코드 축구스타 그 정신적 지주 두 여자 박주영에게 힘 두 여자 사람 아들 기도 어머니 사람 굼벵이 별명 그 여자친구 전반 시즌 마지막 경기 열리 7월 10일 이날 박주영 스무번 생일 이날 관중석 선글래스 경기 관전 한 여인 수 박주영 어머니 것 자신 아들 그림자 존재 이날 박주영 여자친구 지갑 자랑 최근 자신 홈페이지 여자친구 사진 공개 그녀 따뜻한 애정 한편 두 사람 사연 이번 첫 공개 박주영 목표 박지성 박주영 꿈 잉글랜드 진출 것 최근 맨체스터 유나이티드팀 한 박지성 그 꿈 박지성 세계 인정 스타 고등학교때 눈 수 프로팀 대학팀 그 지도한 고등학교 감독 평소 분 명지대 감독 박지성 부탁 정도 그 좌절 력 올림픽대표팀 허정무 감독 눈 리그 진출 2002 한일 월드컵 스타 벤치 그라운드 딩크 감독 감독 부임 네덜란드 아인트호벤팀 박지성 한번 위 홈팬들 아시아 실력 수 노골적 야유 그 부진 1년 반 히딩크 감독 비난 그 출장 박지성 세계적 스타 수 맨유팀 퍼거슨 감독 박지성 선택 것 그 녹초 이상 수 때 수 때문 의지 포기 태극전사 피 그 것 이 우려 벤치신세 대 그라운드 장악하 그 꿈 꿈 두 젊은이 꿈 것 매순간 노력 한국 축구 역사 두 젊은이 두 청년 꿈 순간 우리 그 인생 축구장 우리 좌절 꿈'

In [278]:
get_recommendations(1, cosine_sim2)

380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

In [279]:
get_recommendations(1, cosine_sim2)

380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

In [292]:
print('episode : ' + metadata['epi'][236])
get_recommendations(236, cosine_sim2)

episode : 마지막 선택, 품위 있게 죽고 싶다


324              최악의 시나리오 2부 - 온난화의 마지막 페이지
350     <아동범죄, 미스터리의 과학>1부 아이들은 왜 낯선 사람을 따라
57               29일째 날의 이스터 섬 - 과거로부터의 메시지
644                            몽골에서 지구를 지키다
525                          2009 영상기록 - 기후
1585      생과 사의 강 브라마푸트라 3부 분노의 강, 잃어버린 지평선
403                               대보초의 하얀죽음
352      <아동범죄, 미스터리의 과학>3부 우리아이 어떻게 지킬것인가?
1571                       달의 기적 1부 달이 차오른다
1276       생명 40억년의 비밀 5부 모든 것의 끝 혹은 시작, 멸종
Name: epi, dtype: object

In [281]:
# 결과 이상함................
print('episode : '+ metadata['epi'][1])
get_recommendations(1, cosine_sim2)

episode : 꿈을 쏘다! - 박주영 박지성


380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

## 추천 함수?

In [283]:
def recommender(idx):
    print('episode :'+ str(idx) + ' / ' + metadata['epi'][idx-1])
    
    return get_recommendations(idx, cosine_sim2)
    

In [295]:
get_recommendations(236)

324              최악의 시나리오 2부 - 온난화의 마지막 페이지
350     <아동범죄, 미스터리의 과학>1부 아이들은 왜 낯선 사람을 따라
57               29일째 날의 이스터 섬 - 과거로부터의 메시지
644                            몽골에서 지구를 지키다
525                          2009 영상기록 - 기후
1585      생과 사의 강 브라마푸트라 3부 분노의 강, 잃어버린 지평선
403                               대보초의 하얀죽음
352      <아동범죄, 미스터리의 과학>3부 우리아이 어떻게 지킬것인가?
1571                       달의 기적 1부 달이 차오른다
1276       생명 40억년의 비밀 5부 모든 것의 끝 혹은 시작, 멸종
Name: epi, dtype: object

In [290]:
for i in metadata['idx']:
    print(recommender(i))
    print('')

episode :1 / 그랜드 슬램 대탐험  걸어서 지구 끝까지
380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

episode :2 / 꿈을 쏘다! - 박주영 박지성
233                                         당신은 박지성을 아는가
1113                                     박지성, 오 캡틴 마이 캡틴
334                                   추신수, 마침내 메이저리그를 치다
1243                                      슈틸리케, 리더십을 부탁해
460     <2010 MBC 현대사 연속 기획>경술국치 100년 특집“축구 그리고 세 개의 조국”
27                                 ‘D-103, 축구 국가대표 전훈기록’
1462                           리우올림픽 특집 박세리, 내 생에 최고의 순간

Name: epi, dtype: object

episode :71 / SBS 스페셜 - 나는 나, 재일 동포 3,4세들의 새로운 선택
460     <2010 MBC 현대사 연속 기획>경술국치 100년 특집“축구 그리고 세 개의 조국”
461                     한일병탄 100년 특집 [역사전쟁] - 1부 만들어진 역사
45                 8.15 특집 SBS스페셜 - 일본 청년 아오키의 끝나지 않은 전쟁
957                                  젠야(前夜) - 열도의 위험한 밤 
557                                          츠카 코헤이와 김봉웅
463                  한일병탄 100년 특집 [역사전쟁] - 2부 역사의 역설(逆說)
1366                   3.1절 특집 일본의 또 다른 얼굴, 카운터스 행동대(男組)
1725                                 #미투 (Me Too) 나는 말한다
1128                               ‘유홍준, 일본 속 한국을 걷다’ 1부
1131                               ‘유홍준, 일본 속 한국을 걷다’ 2부
Name: epi, dtype: object

episode :72 / 미각예찬  당신의 혀를 보호하라
935                              입맛의 역습
1310                  맛이란 무엇인가 3부 맛의 교육
1252              균이 당신을 지배한다 - 세균숲 이야기
1042                             잠과의 전쟁
156              당신의 과학 - 제1부 당신은 누구세요?
157             당신의 과학 - 제2부 당신의 행복한 시간
993                           물 한 잔의 기적
25              

Name: epi, dtype: object

episode :166 / 말도 아리랑 
215                             민간인 통제구역
271                            연평별곡 (別曲)
1880     방송의 날 특집6개월 후 만납시다: 북한 결핵병원 이야기
1767           누가 북한을 움직이는가(1)평양의 파워 엘리트
1790          남북정상회담 특집 다큐 <테이블 너머의 김정은>
98      남측 벼를 심으려 왔습니다 - 평양, 당곡리에서 맺은 결실
134                           거꾸로 가는 도시 
948                        최초공개 북한의 백두대간
1640                           김정은의 두 얼굴
395                     2010 JSA 그곳의 이야기
Name: epi, dtype: object

episode :167 / 물위를 떠도는 영혼 바다 집시
474                          불가사리와의 전쟁
937                     마지막 해인-오랑바자우라우
301     아시아 원시부족 탐험 - 2부 바다의 방랑자, 바자우족
1852      아시아 원시 부족 탐험 - 바다의 방랑자, 바자우족
403                          대보초의 하얀죽음
85                         심해, 마지막 머구리
452       대한민국 해양대탐험 - 1부 꿈의 좌표를 향한 도전
1869       진화와 공존의 섬 갈라파고스 제 2부 혼돈의 바다
1222              영산도 섬소년 이야기 - 바다의 노래
1777         백성의 물고기 - 3부 행복한 비린내, 고등어
Name: epi, dtype: object

episode :168 / 기억 속으로 떠나는 여행
1643               오늘을 잊지 마 - 사라진 25살의

Name: epi, dtype: object

episode :235 / 방랑식객 (放浪食客)
1629                       식객들의 식탐
289     방랑식객 (放浪食客) 2. “ 올레길을 가다 ”
473                 방랑식객 5. 일본을 가다
1613          옴니버스食 다큐멘터리 더 잘 먹는 법
1621                  진관사, 절밥 한 그릇
1034                    이영애의 만찬 2부
422      방랑식객 (放浪食客) 4  아이들의 봄을 찾다
1309             맛이란 무엇인가 2부 맛의 기억
1273                   요리, 남자를 바꾸다
1678               남자, 요리와 사랑에 빠지다
Name: epi, dtype: object

episode :236 / 가라앉는 지구
324              최악의 시나리오 2부 - 온난화의 마지막 페이지
350     <아동범죄, 미스터리의 과학>1부 아이들은 왜 낯선 사람을 따라
57               29일째 날의 이스터 섬 - 과거로부터의 메시지
644                            몽골에서 지구를 지키다
525                          2009 영상기록 - 기후
1585      생과 사의 강 브라마푸트라 3부 분노의 강, 잃어버린 지평선
403                               대보초의 하얀죽음
352      <아동범죄, 미스터리의 과학>3부 우리아이 어떻게 지킬것인가?
1571                       달의 기적 1부 달이 차오른다
1276       생명 40억년의 비밀 5부 모든 것의 끝 혹은 시작, 멸종
Name: epi, dtype: object

episode :237 / 마지막 선택, 품위 있게 죽고 싶다
13                                         유언 -

Name: epi, dtype: object

episode :299 / Celebrity Biography - 박찬호 편
143        결혼안식휴가 - 제1부 부부, 재발견하다
92        2007 남성보고서 男子는 무엇으로 사는가
1665                       승엽의 시대
743       나는 산다, 김성근 “9회말까지 인생이다”
440                 나는 한국남자와 결혼했다
334            추신수, 마침내 메이저리그를 치다
24      한국을 떠난 사람들 - 그들이 말하는 대한민국
164          길을 찾는 사람들 - 휴메니버시티 -
1814           행복한 부부생활을 위한 졸혼 연습
1495       ‘우리가 결혼하지 않는 진짜 이유’ 1부
Name: epi, dtype: object

episode :300 / 매력 DNA, 그들이 인기있는 이유
430               춤, 세상을 흔들다-1부 매력 D.N.A, 춤
347                               길, 매력을 팔다
26           이제는 웃다가 성공한다 - 진수테리 fun경영 공개특강
350     <아동범죄, 미스터리의 과학>1부 아이들은 왜 낯선 사람을 따라
538                       헐리웃 키즈, 한국의 끼로 날다
159                     SBS스페셜 다큐멘터리 동화 네팔 
164                    길을 찾는 사람들 - 휴메니버시티 -
407                 당신의 성격 3부 나는 내성적인 사람입니다
1047                             공간 혁명 작은 집
750                          동물, 행복의 조건 1부 
Name: epi, dtype: object

episode :301 / 아시아 원시부족 탐험 - 1부 아시아의 흑진주, 두마카트족
1851     아시아 원시 

...

Name: epi, dtype: object

episode :1469 / 앙트레프레너, 경제강국의 비밀 3부 부(富)의 법칙
1472             앙트레프레너, 경제강국의 비밀 4부 왕들의 선택
1475     앙트레프레너, 경제강국의 비밀 6부 앙트레프레너를 키우는 나라
1474       앙트레프레너, 경제강국의 비밀 5부 아메리칸 시스템의 비밀
1465           앙트레프레너, 경제강국의 비밀 1부 창조적 파괴자들
1467           앙트레프레너, 경제강국의 비밀 2부 부자나라의 탄생
1836         <5원소, 문명의 기원> 4부 철, 인류의 견고한 욕망
477       EBS 기획특집 국가경쟁력 리포트 제3부 문화강국이 되는 길
1693    <4차 산업혁명, 교육 패러다임의 대전환> - 1부 대학의 변신
111                             자연다큐멘터리 <흙>
135                            자연주의 마을 토트네스
Name: epi, dtype: object

episode :1470 / 운명처럼 - 생명의 전선에서
527                   짝 2부 - 너는 내 운명인가?
4                 메이드 인 코리아 3부 - 한국인의 손
1162             한국음식을 말하다 2부 - 밥상위의 근대
1160          한국음식을 말하다 1부 - 오래된 것과의 만남
1163    한국음식을 말하다 3부 - 과식의 시대에서 미식의 시대로
1779             고독한 우주인 - 지구 귀환 3649일째
890                     그가 남긴 사랑, 카라바이요
531                   짝 3부 - 미워도 다시 한 번
926           [위대한 로마] 황제들의 정치무대 - 콜로세움
1516                                 탄핵
Name: epi, dtype: object

episo

episode :1579 / 아빠가 임신했다
702               “아기를 원하십니까-BABY PLAN”
1369        아빠 찾는 게 죄인가요? - 코피노의 마지막 선택
1079            아버지의 성 1부 부성 본능의 불이 켜지다
323                           마흔에 엄마 되다
1803                           앵그리맘의 반격
143              결혼안식휴가 - 제1부 부부, 재발견하다
36                   新육아 보고서 - 父性의 재발견 
1583    인구절벽 원년 보고서 ?2부- <1.17 기적의 출산율>
699                   “나는 산다 - 오복식 고물상”
144               결혼안식휴가 - 제2부 나를 기억해내다
Name: epi, dtype: object

episode :1580 / 인구절벽 원년 보고서 1부 <2년제 인생 ? 결혼 못 하는 청춘>
1499     ‘우리가 결혼하지 않는 진짜 이유’ 2부
1294     결혼의 진화 - 2부 결혼, 하셨습니까?
1495     ‘우리가 결혼하지 않는 진짜 이유’ 1부
1292       결혼의 진화 - 1부 인류, 결혼하다
1295     결혼의 진화 - 3부 새로운 가족이 온다
1487    부부는 무엇으로 사는가, 1부 부부의 탄생
740                 세 여자의 웨딩드레스
1197        연애만 8년째, 결혼할 수 있을까?
128             인터뷰다큐 성장통, '만남'
1702              시골에 가게를 차렸습니다
Name: epi, dtype: object

episode :1581 / 아프리카로 간 선생님들호이 - 10년간의 기록
1657                       스팀 교육 - 미래 인재를 말하다
1574                              대2병, 학교를 묻다
1359    교육패러다임 대

Name: epi, dtype: object

episode :1677 / 불멸의 마야 - 2부 260일 달력, 촐낀
1675              불멸의 마야 - 1부 옥수수 문명을 찾아서
1677              불멸의 마야 - 3부 인디오는 어디로 가나
603          인류문명탐험-1부 사라진 고대무역도시, 인더스 문명
1837     <5원소, 문명의 기원> 5부 불, 문명과 야만의 두 얼굴
608         인류문명탐험-4부 현대 서구문명의 뿌리, 그리스 문명
170     문명의3대교차로를가다-3부 동서양의 아름다운 만남, 이스탄불
604        인류문명탐험-2부 사막 위에 꽃을 피우다, 이집트 문명
1833           <5원소, 문명의 기원> 2부 흙, 생명을 품다
1119                    문명과 수학 - 1부 수의 시작
169      문명의3대교차로를가다-2부 알함브라의 추억,스페인안달루시아
Name: epi, dtype: object

episode :1678 / 불멸의 마야 - 3부 인디오는 어디로 가나
1676              불멸의 마야 - 2부 260일 달력, 촐낀
1675              불멸의 마야 - 1부 옥수수 문명을 찾아서
603          인류문명탐험-1부 사라진 고대무역도시, 인더스 문명
1837     <5원소, 문명의 기원> 5부 불, 문명과 야만의 두 얼굴
170     문명의3대교차로를가다-3부 동서양의 아름다운 만남, 이스탄불
608         인류문명탐험-4부 현대 서구문명의 뿌리, 그리스 문명
169      문명의3대교차로를가다-2부 알함브라의 추억,스페인안달루시아
1833           <5원소, 문명의 기원> 2부 흙, 생명을 품다
168     문명의3대교차로를가다-1부 찬란한 중세 이슬람의 추억,모로코
1832          <5원소, 문명의 기원> 1부 나무, 지식을 담다
Name: epi, dtype: object

ep

Name: epi, dtype: object

episode :1746 / <4차 인간> 2부 인간은 기계인가?
1746              <4차 인간> 3부 어떻게 기계와 공존할 것인가?
1744                <4차 인간> 1부 우리는 영원할 수 있을까?
981                                     인간과 개
1308                      맛이란 무엇인가 - 1부 맛의 비밀
1842                    플라스틱 지구(1) - 플라스틱의 역습
1649    <4차 산업혁명시대 교육대혁명> 1부 AI와 인간의 공존은 가능한가
1293                           로봇, 인간을 구할 것인가
328                         인간과 고양이 2부-고양이 전쟁
1654                      <철학하라> 제2부 - 마음의 조건
1751                  가축 - 2부 사치스러운 음식, 젖과 고기
Name: epi, dtype: object

episode :1747 / <4차 인간> 3부 어떻게 기계와 공존할 것인가?
1745                     <4차 인간> 2부 인간은 기계인가?
1744                <4차 인간> 1부 우리는 영원할 수 있을까?
981                                     인간과 개
1649    <4차 산업혁명시대 교육대혁명> 1부 AI와 인간의 공존은 가능한가
1308                      맛이란 무엇인가 - 1부 맛의 비밀
1293                           로봇, 인간을 구할 것인가
1654                      <철학하라> 제2부 - 마음의 조건
1842                    플라스틱 지구(1) - 플라스틱의 역습
653                언어발달의 수수께끼 3부-나도 말을 

Name: epi, dtype: object

episode :1838 / <5원소, 문명의 기원> 5부 불, 문명과 야만의 두 얼굴
1833           <5원소, 문명의 기원> 2부 흙, 생명을 품다
1832          <5원소, 문명의 기원> 1부 나무, 지식을 담다
1835           <5원소, 문명의 기원> 3부 물, 위대한 순환
1836       <5원소, 문명의 기원> 4부 철, 인류의 견고한 욕망
170     문명의3대교차로를가다-3부 동서양의 아름다운 만남, 이스탄불
603          인류문명탐험-1부 사라진 고대무역도시, 인더스 문명
1553                       <사라진 인류> 1부 멸종
168     문명의3대교차로를가다-1부 찬란한 중세 이슬람의 추억,모로코
1675              불멸의 마야 - 1부 옥수수 문명을 찾아서
1119                    문명과 수학 - 1부 수의 시작
Name: epi, dtype: object

episode :1839 / 식탁 위로 돌아 온 미세 플라스틱
1842           플라스틱 지구(1) - 플라스틱의 역습
1843           플라스틱 지구(2) - 굿바이 플라스틱
915      [과학 대기획-다섯개의 열쇠]2부-신소재(新素材)
474                        불가사리와의 전쟁
85                       심해, 마지막 머구리
30            작은 나라의 큰 행복 - 은둔의 땅 부탄
403                        대보초의 하얀죽음
1252           균이 당신을 지배한다 - 세균숲 이야기
48         빛에 관한 보고서 - 독이 되고 약이 되는 빛
1317    우리 WE, 3부 우리는 왜 우리가 되지 못하는가?
Name: epi, dtype: object

episode :1840 / 생선의 종말 1부 신이 된 물고기, 참치
1840     

for i in metadata['idx']:
    print(recommender(i))
    print('')
